# Symbolic expression for the amplitude model

$$
\begin{eqnarray}
A^{12} &=& \frac{\sum a_m Y_2^m (\Omega_1)}{s_{12}-m^2_{a_2}+im_{a_2} \Gamma_{a_2}} \\
A^{23} &=& \frac{\sum b_m Y_1^m (\Omega_2)}{s_{23}-m^2_{\Delta}+im_{\Delta} \Gamma_{\Delta}}  \\
\
A^{31} &=& \frac{c_0}{s_{31}-m^2_{N^*}+im_{N^*} \Gamma_{N^*}}
\end{eqnarray}
$$ (BW_SH_label)

In [ ]:
import numpy as np
import graphviz
import sympy as sp
from sympy import (
    re,
    im,
    I,
    E,
    symbols,
    Sum,
    IndexedBase,
    Function,
    Ynm,
    Symbol,
    simplify,
)
from IPython.display import display, Latex
import matplotlib.pyplot as plt

# $A^{12}$

In [ ]:
newa = IndexedBase("a")
type(newa[0])

In [ ]:
s12, m_a2, Gamma_a2 = symbols(r"s_{12} m_{a_2} \Gamma_{a_2}")
l, m, theta1, phi1 = symbols("l m theta_1 phi_1")
# Ynm(l, m, theta, phi)
a = IndexedBase("a")
# Y2_m = Function("Y_2^m")("Omega_1")
m = symbols("m", cls=sp.Idx)
# Omega1 = symbols("Omega_1")

A12 = sp.Sum(a[m] * Ynm(2, m, theta1, phi1), (m, -2, 2)) / (
    s12 - m_a2**2 + I * m_a2 * Gamma_a2
)
A12

In [ ]:
Ynm(2, 1, 1, 1).expand(func=True)

In [ ]:
A12.free_symbols

In [ ]:
A12_func = sp.lambdify(
    [s12, a[-2], a[-1], a[-0], a[1], a[2], m_a2, Gamma_a2, theta1, phi1],
    A12.doit().expand(func=True),
)
A12_func

In [ ]:
# Example values
m_a2_val = 1.5
Gamma_a2_val = 0.6
a_vals = [1, 2, 3, 4, 5]

# Generate s values
s_values = np.linspace(0, 5, 100)
# PHI = np.linspace(-np.pi, +np.pi, num=100)
# THETA = np.linspace(0, np.pi, num=100)

PHI, THETA = np.meshgrid(
    np.linspace(-np.pi, +np.pi, num=100),
    np.linspace(0, np.pi, num=100),
)
A12_values = A12_func(s_values, *a_vals, m_a2_val, Gamma_a2_val, THETA, PHI)

# Create the plot
plt.figure(figsize=(10, 6))

plt.plot(s_values, A12_values.imag, label="Imaginary Part", linestyle="-", color="b")
plt.plot(s_values, A12_values.real, label="Real Part", linestyle="--", color="r")
plt.plot(s_values, np.abs(A12_values) ** 2, label="|A12|^2", linestyle="-.", color="g")
plt.plot(
    s_values, np.angle(A12_values), label="Phase (angle)", linestyle=":", color="m"
)

plt.xlabel("s")
plt.ylabel(r"$A^{12}$ components")
plt.title(r"Components of $A^{12}$ vs s")
# plt.legend()
# plt.grid(True)
plt.show()

In [ ]:
# Example values
m_a2_val = 1.5
Gamma_a2_val = 0.6
a_vals = [1, 2, 3, 4, 5]

# Generate s values
s_values = np.linspace(0, 5, 100)
# PHI = np.linspace(-np.pi, +np.pi, num=100)
# THETA = np.linspace(0, np.pi, num=100)
PHI = np.pi / 4
THETA = np.pi / 4
# PHI, THETA = np.meshgrid(
#     np.linspace(-np.pi, +np.pi, num=100),
#     np.linspace(0, np.pi, num=100),
# )
A12_values = A12_func(s_values, *a_vals, m_a2_val, Gamma_a2_val, THETA, PHI)

# Create the plot
plt.figure(figsize=(10, 6))

plt.plot(s_values, A12_values.imag, label="Imaginary Part", linestyle="-", color="b")
plt.plot(s_values, A12_values.real, label="Real Part", linestyle="--", color="r")
plt.plot(s_values, np.abs(A12_values) ** 2, label="|A12|^2", linestyle="-.", color="g")
plt.plot(
    s_values, np.angle(A12_values), label="Phase (angle)", linestyle=":", color="m"
)

plt.xlabel("s")
plt.ylabel(r"$A^{12}$ components")
plt.title(r"Components of $A^{12}$ vs s at 45 degree")
plt.legend()
# plt.grid(True)
plt.show()

# $A^{23}$

In [ ]:
s23, m_Delta, Gamma_Delta = symbols("s_{23} m_Delta Gamma_Delta")
b = IndexedBase("b")
Y1_m = Function("Y_1^m")("Omega_2")
theta2, phi2 = symbols("theta_2 phi_2")
A23 = Sum(b[m] * Ynm(1, m, theta2, phi2), (m, -1, 1)) / (
    s23 - m_Delta**2 + I * m_Delta * Gamma_Delta
)
A23

In [ ]:
A23.free_symbols

In [ ]:
# A23_func = sp.lambdify([s23, b_m, m_Delta, Gamma_Delta, theta2, phi2], A23)

# $A^{31}$

In [ ]:
s31, c0, m_Nstar, Gamma_Nstar = symbols(r"s_{31} c_0 m_{N^*} \Gamma_{N^*}")
A31 = c0 / (s31 - m_Nstar**2 + I * m_Nstar * Gamma_Nstar)
A31

In [ ]:
A31.free_symbols

In [ ]:
sp.lambdify([s31, c0, m_Nstar, Gamma_Nstar], A31)

In [ ]:
A31_func = sp.lambdify([s31, c0, m_Nstar, Gamma_Nstar], A31)

In [ ]:
s_values = np.linspace(0, 5, 100)
A31_values = A31_func(s_values, 1, 1.5, 0.6)
plt.plot(s_values, A31_values.imag)
plt.plot(s_values, A31_values.real)
plt.plot(s_values, np.abs(A31_values) ** 2)
plt.plot(s_values, np.angle(A31_values))

In [ ]:
# Example values for m_Nstar and Gamma_Nstar
m_Nstar_val = 1.5
Gamma_Nstar_val = 0.6
c0_val = 1

# Generate s values
s_values = np.linspace(0, 5, 100)
A31_values = A31_func(s_values, c0_val, m_Nstar_val, Gamma_Nstar_val)

# Create the plot
plt.figure(figsize=(10, 6))

plt.plot(s_values, A31_values.imag, label="Imaginary Part", linestyle="-", color="b")
plt.plot(s_values, A31_values.real, label="Real Part", linestyle="--", color="r")
plt.plot(s_values, np.abs(A31_values) ** 2, label="|A31|^2", linestyle="-.", color="g")
plt.plot(
    s_values, np.angle(A31_values), label="Phase (angle)", linestyle=":", color="m"
)

plt.xlabel("s")
plt.ylabel(r"$A^{31}$ components")
plt.title(r"Components of $A^{31}$ vs s")
plt.legend()
# plt.grid(True)
plt.show()

In [ ]:
plt.plot(A31_values.real, A31_values.imag)
plt.title(r"Argand diagram of $A^{31}$")
plt.show()

In [ ]:
# # Example values for m_Nstar and Gamma_Nstar
# m_Nstar_val = 1.5
# Gamma_Nstar_val = 0.6
# c0_val = 1

# # Generate s values
# s_values = np.linspace(0, 5, 100)
# A12_values = A12_func(s_values, c0_val, m_Nstar_val, Gamma_Nstar_val)

# # Create the plot
# plt.figure(figsize=(10, 6))

# plt.plot(s_values, A31_values.imag, label="Imaginary Part", linestyle="-", color="b")
# plt.plot(s_values, A31_values.real, label="Real Part", linestyle="--", color="r")
# plt.plot(s_values, np.abs(A31_values) ** 2, label="|A31|^2", linestyle="-.", color="g")
# plt.plot(
#     s_values, np.angle(A31_values), label="Phase (angle)", linestyle=":", color="m"
# )

# plt.xlabel("s")
# plt.ylabel(r"$A^{31}$ components")
# plt.title(r"Components of $A^{31}$ vs s")
# plt.legend()
# plt.grid(True)
# plt.show()